In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
import sqlite3
import psycopg2
import json
import os, zipfile
from bs4 import BeautifulSoup

from IPython.core.display import HTML
pd.options.display.max_columns = 100
import datetime
import requests

from sqlalchemy import create_engine
from difflib import SequenceMatcher, get_close_matches
from IPython.core.display import HTML
from ua_parser import user_agent_parser
import seaborn as sb
import pprint
pp = pprint.PrettyPrinter(indent=4)

HTML("<style>.container { width:100% !important; }</style>")

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(open("data_verkiezingsuitslagen/index.htm"), "html.parser")


## Download all ZIP files

In [31]:
for i, a in enumerate(soup.findAll('a')):
    #if(i>10): break
    link = a.get('href')
    if not re.search('ZIP', link): continue
    full_link = 'https://2019.elections.openknowledge.be/live/' + link
    
    r = requests.get(full_link)
    
    filename = 'data_verkiezingsuitslagen/zipfiles/'+link
    with open(filename, "wb") as handle:
        for data in r.iter_content():
            handle.write(data)

## Unzip all ZIP files

In [36]:
for file in os.listdir('data_verkiezingsuitslagen/zipfiles/'):
    if not re.search('ZIP', file): continue
    print('Unzizipping {}'.format(file))
    zip_ref = zipfile.ZipFile('data_verkiezingsuitslagen/zipfiles/' + file, 'r')
    zip_ref.extractall('data_verkiezingsuitslagen/xmls/')
    zip_ref.close()

Unzizipping E_0001_000_0001_WL.ZIP
Unzizipping E_0002_000_0002_WL.ZIP
Unzizipping E_0003_000_0003_WL.ZIP
Unzizipping E_0010_000_0001_BR.ZIP
Unzizipping E_0010_000_0001_DE.ZIP
Unzizipping E_0013_000_0004_WL.ZIP
Unzizipping E_0020_000_0001_VL.ZIP
Unzizipping E_0020_000_0002_BR.ZIP
Unzizipping E_0020_000_0002_DE.ZIP
Unzizipping E_0023_000_0005_WL.ZIP
Unzizipping E_0024_000_0006_WL.ZIP
Unzizipping E_0030_000_0002_VL.ZIP
Unzizipping E_0030_000_0003_BR.ZIP
Unzizipping E_0030_000_0003_DE.ZIP
Unzizipping E_0034_000_0007_WL.ZIP
Unzizipping E_0035_000_0008_WL.ZIP
Unzizipping E_0037_000_0009_WL.ZIP
Unzizipping E_0040_000_0004_BR.ZIP
Unzizipping E_0040_000_0004_DE.ZIP
Unzizipping E_0047_000_0010_WL.ZIP
Unzizipping E_0048_000_0011_WL.ZIP
Unzizipping E_0049_000_0012_WL.ZIP
Unzizipping E_0050_000_0003_VL.ZIP
Unzizipping E_0050_000_0005_BR.ZIP
Unzizipping E_0050_000_0013_WL.ZIP
Unzizipping E_0050_001_0005_DE.ZIP
Unzizipping E_0051_000_0014_WL.ZIP
Unzizipping E_0057_002_0006_DE.ZIP
Unzizipping E_0057_0

Unzizipping E_1039_052_0118_WL.ZIP
Unzizipping E_1047_052_0119_WL.ZIP
Unzizipping E_1055_053_0120_WL.ZIP
Unzizipping E_1064_055_0121_WL.ZIP
Unzizipping E_1066_012_0025_CK.ZIP
Unzizipping E_1067_055_0122_WL.ZIP
Unzizipping E_1071_055_0123_WL.ZIP
Unzizipping E_1074_001_0029_VL.ZIP
Unzizipping E_1074_055_0124_WL.ZIP
Unzizipping E_1076_012_0026_CK.ZIP
Unzizipping E_1079_055_0125_WL.ZIP
Unzizipping E_1085_056_0126_WL.ZIP
Unzizipping E_1091_056_0127_WL.ZIP
Unzizipping E_1096_057_0128_WL.ZIP
Unzizipping E_1102_059_0129_WL.ZIP
Unzizipping E_1110_063_0130_WL.ZIP
Unzizipping E_1113_063_0131_WL.ZIP
Unzizipping E_1114_001_0030_VL.ZIP
Unzizipping E_1122_065_0132_WL.ZIP
Unzizipping E_1126_012_0027_CK.ZIP
Unzizipping E_1131_065_0133_WL.ZIP
Unzizipping E_1134_065_0134_WL.ZIP
Unzizipping E_1141_065_0135_WL.ZIP
Unzizipping E_1144_001_0031_VL.ZIP
Unzizipping E_1149_066_0136_WL.ZIP
Unzizipping E_1156_012_0028_CK.ZIP
Unzizipping E_1157_068_0137_WL.ZIP
Unzizipping E_1166_068_0138_WL.ZIP
Unzizipping E_1171_0

Unzizipping E_3330_031_0097_VL.ZIP
Unzizipping E_3341_031_0098_VL.ZIP
Unzizipping E_3350_031_0099_VL.ZIP
Unzizipping E_3354_038_0094_CK.ZIP
Unzizipping E_3381_032_0100_VL.ZIP
Unzizipping E_3386_038_0095_CK.ZIP
Unzizipping E_3404_033_0101_VL.ZIP
Unzizipping E_3420_034_0102_VL.ZIP
Unzizipping E_3446_034_0103_VL.ZIP
Unzizipping E_3472_040_0096_CK.ZIP
Unzizipping E_3492_034_0104_VL.ZIP
Unzizipping E_3514_034_0105_VL.ZIP
Unzizipping E_3533_041_0097_CK.ZIP
Unzizipping E_3557_035_0106_VL.ZIP
Unzizipping E_3587_035_0107_VL.ZIP
Unzizipping E_3613_037_0108_VL.ZIP
Unzizipping E_3620_043_0098_CK.ZIP
Unzizipping E_3649_043_0099_CK.ZIP
Unzizipping E_3650_038_0109_VL.ZIP
Unzizipping E_3683_038_0110_VL.ZIP
Unzizipping E_3698_044_0100_CK.ZIP
Unzizipping E_3716_038_0111_VL.ZIP
Unzizipping E_3735_038_0112_VL.ZIP
Unzizipping E_3750_046_0101_CK.ZIP
Unzizipping E_3759_038_0113_VL.ZIP
Unzizipping E_3799_039_0114_VL.ZIP
Unzizipping E_3807_039_0115_VL.ZIP
Unzizipping E_3821_048_0102_CK.ZIP
Unzizipping E_3834_0

Unzizipping E_5973_142_0181_CK.ZIP
Unzizipping E_5999_144_0182_CK.ZIP
Unzizipping E_6019_147_0183_CK.ZIP
Unzizipping E_6024_147_0184_CK.ZIP
Unzizipping E_6039_151_0185_CK.ZIP
Unzizipping E_6054_152_0186_CK.ZIP
Unzizipping E_6075_155_0187_CK.ZIP
Unzizipping E_6085_156_0188_CK.ZIP
Unzizipping E_6094_157_0189_CK.ZIP
Unzizipping E_6115_158_0190_CK.ZIP
Unzizipping E_6132_161_0191_CK.ZIP
Unzizipping E_6153_162_0192_CK.ZIP
Unzizipping E_6159_163_0193_CK.ZIP
Unzizipping E_6177_163_0194_CK.ZIP
Unzizipping E_6194_166_0195_CK.ZIP
Unzizipping E_6204_167_0196_CK.ZIP
Unzizipping E_6212_169_0197_CK.ZIP
Unzizipping E_6218_169_0198_CK.ZIP
Unzizipping E_6235_170_0199_CK.ZIP
Unzizipping E_6244_172_0200_CK.ZIP
Unzizipping E_6250_176_0201_CK.ZIP
Unzizipping E_6255_176_0202_CK.ZIP
Unzizipping E_6264_176_0203_CK.ZIP
Unzizipping E_6270_177_0204_CK.ZIP
Unzizipping E_6277_178_0205_CK.ZIP
Unzizipping E_6301_178_0206_CK.ZIP
Unzizipping E_6310_178_0207_CK.ZIP
Unzizipping E_6324_178_0208_CK.ZIP
Unzizipping E_6330_1

Unzizipping E_7324_225_0087_EU.ZIP
Unzizipping E_7324_225_0088_EU.ZIP
Unzizipping F0000_000_0000_BR.ZIP
Unzizipping F0000_000_0000_CK.ZIP
Unzizipping F0000_000_0000_DE.ZIP
Unzizipping F0000_000_0000_EU.ZIP
Unzizipping F0000_000_0000_SE.ZIP
Unzizipping F0000_000_0000_VL.ZIP
Unzizipping F0000_000_0000_WL.ZIP
Unzizipping F0057_002_0010_DE.ZIP
Unzizipping F6864_174_0002_EU.ZIP
Unzizipping R0002_000_0002_WL.ZIP
Unzizipping R0003_000_0003_WL.ZIP
Unzizipping R0010_000_0001_BR.ZIP
Unzizipping R0010_000_0001_DE.ZIP
Unzizipping R0013_000_0004_WL.ZIP
Unzizipping R0020_000_0002_BR.ZIP
Unzizipping R0020_000_0002_DE.ZIP
Unzizipping R0023_000_0005_WL.ZIP
Unzizipping R0024_000_0006_WL.ZIP
Unzizipping R0030_000_0003_BR.ZIP
Unzizipping R0030_000_0003_DE.ZIP
Unzizipping R0034_000_0007_WL.ZIP
Unzizipping R0035_000_0008_WL.ZIP
Unzizipping R0037_000_0009_WL.ZIP
Unzizipping R0040_000_0004_BR.ZIP
Unzizipping R0040_000_0004_DE.ZIP
Unzizipping R0047_000_0010_WL.ZIP
Unzizipping R0048_000_0011_WL.ZIP
Unzizipping 

Unzizipping R1014_001_0028_VL.ZIP
Unzizipping R1014_047_0115_WL.ZIP
Unzizipping R1023_048_0116_WL.ZIP
Unzizipping R1028_049_0117_WL.ZIP
Unzizipping R1036_012_0024_CK.ZIP
Unzizipping R1039_052_0118_WL.ZIP
Unzizipping R1047_052_0119_WL.ZIP
Unzizipping R1055_053_0120_WL.ZIP
Unzizipping R1064_055_0121_WL.ZIP
Unzizipping R1066_012_0025_CK.ZIP
Unzizipping R1067_055_0122_WL.ZIP
Unzizipping R1071_055_0123_WL.ZIP
Unzizipping R1074_001_0029_VL.ZIP
Unzizipping R1074_055_0124_WL.ZIP
Unzizipping R1076_012_0026_CK.ZIP
Unzizipping R1079_055_0125_WL.ZIP
Unzizipping R1085_056_0126_WL.ZIP
Unzizipping R1091_056_0127_WL.ZIP
Unzizipping R1096_057_0128_WL.ZIP
Unzizipping R1102_059_0129_WL.ZIP
Unzizipping R1110_063_0130_WL.ZIP
Unzizipping R1113_063_0131_WL.ZIP
Unzizipping R1114_001_0030_VL.ZIP
Unzizipping R1122_065_0132_WL.ZIP
Unzizipping R1126_012_0027_CK.ZIP
Unzizipping R1131_065_0133_WL.ZIP
Unzizipping R1134_065_0134_WL.ZIP
Unzizipping R1141_065_0135_WL.ZIP
Unzizipping R1144_001_0031_VL.ZIP
Unzizipping R1

Unzizipping R3341_031_0098_VL.ZIP
Unzizipping R3350_031_0099_VL.ZIP
Unzizipping R3354_038_0094_CK.ZIP
Unzizipping R3381_032_0100_VL.ZIP
Unzizipping R3386_038_0095_CK.ZIP
Unzizipping R3404_033_0101_VL.ZIP
Unzizipping R3420_034_0102_VL.ZIP
Unzizipping R3446_034_0103_VL.ZIP
Unzizipping R3472_040_0096_CK.ZIP
Unzizipping R3492_034_0104_VL.ZIP
Unzizipping R3514_034_0105_VL.ZIP
Unzizipping R3533_041_0097_CK.ZIP
Unzizipping R3557_035_0106_VL.ZIP
Unzizipping R3587_035_0107_VL.ZIP
Unzizipping R3613_037_0108_VL.ZIP
Unzizipping R3620_043_0098_CK.ZIP
Unzizipping R3649_043_0099_CK.ZIP
Unzizipping R3650_038_0109_VL.ZIP
Unzizipping R3683_038_0110_VL.ZIP
Unzizipping R3698_044_0100_CK.ZIP
Unzizipping R3716_038_0111_VL.ZIP
Unzizipping R3735_038_0112_VL.ZIP
Unzizipping R3750_046_0101_CK.ZIP
Unzizipping R3759_038_0113_VL.ZIP
Unzizipping R3799_039_0114_VL.ZIP
Unzizipping R3807_039_0115_VL.ZIP
Unzizipping R3821_048_0102_CK.ZIP
Unzizipping R3834_040_0116_VL.ZIP
Unzizipping R3856_049_0103_CK.ZIP
Unzizipping R3

Unzizipping R6094_157_0189_CK.ZIP
Unzizipping R6115_158_0190_CK.ZIP
Unzizipping R6132_161_0191_CK.ZIP
Unzizipping R6153_162_0192_CK.ZIP
Unzizipping R6159_163_0193_CK.ZIP
Unzizipping R6177_163_0194_CK.ZIP
Unzizipping R6194_166_0195_CK.ZIP
Unzizipping R6204_167_0196_CK.ZIP
Unzizipping R6212_169_0197_CK.ZIP
Unzizipping R6218_169_0198_CK.ZIP
Unzizipping R6235_170_0199_CK.ZIP
Unzizipping R6244_172_0200_CK.ZIP
Unzizipping R6250_176_0201_CK.ZIP
Unzizipping R6255_176_0202_CK.ZIP
Unzizipping R6264_176_0203_CK.ZIP
Unzizipping R6270_177_0204_CK.ZIP
Unzizipping R6277_178_0205_CK.ZIP
Unzizipping R6301_178_0206_CK.ZIP
Unzizipping R6310_178_0207_CK.ZIP
Unzizipping R6324_178_0208_CK.ZIP
Unzizipping R6330_178_0209_CK.ZIP
Unzizipping R6337_178_0210_CK.ZIP
Unzizipping R6343_179_0211_CK.ZIP
Unzizipping R6347_179_0212_CK.ZIP
Unzizipping R6353_180_0213_CK.ZIP
Unzizipping R6358_180_0214_CK.ZIP
Unzizipping R6364_180_0215_CK.ZIP
Unzizipping R6369_181_0216_CK.ZIP
Unzizipping R6375_181_0217_CK.ZIP
Unzizipping R6

## Understand filenames

- Enkel files van de Kamer: CK
- Geen tussentijdse bestanden, dus geen _[0-9][0-9][0-9]_ in de filename, zie handleiding
- Dit zijn allemaal bestanden van Kanton hoofdbureaus

- Enkel NL bestanden, geen tweetalige bestanden. _NL.xml in de filename, en niet _NL_FR.xml of _FR_NL.xml

In [55]:
interesting_files = []
for file in os.listdir('data_verkiezingsuitslagen/xmls/'):
    if not re.search('EML', file): continue
    #if re.search('_NL_FR.EML', file) or re.search('_FR_NL.EML', file): continue
    if not re.search('_CK', file): continue
    if  re.search('_[0-9][0-9][0-9]_', file): continue
    print(file)
    interesting_files.append(file)

1K10999_CK.EML
1K11002_CK.EML
1K11005_CK.EML
1K11009_CK.EML
1K11023_CK.EML
1K11024_CK.EML
1K11054_CK.EML
1K12009_CK.EML
1K12014_CK.EML
1K12021_CK.EML
1K12025_CK.EML
1K12041_CK.EML
1K13001_CK.EML
1K13011_CK.EML
1K13014_CK.EML
1K13025_CK.EML
1K13040_CK.EML
1K13049_CK.EML
1K21001_FR_CK.EML
1K21004_FR_CK.EML
1K21009_FR_CK.EML
1K21012_FR_CK.EML
1K21013_FR_CK.EML
1K21014_FR_CK.EML
1K21015_FR_CK.EML
1K21016_FR_CK.EML
1K21998_FR_CK.EML
1K21999_FR_CK.EML
1K23002_CK.EML
1K23027_CK.EML
1K23050_CK.EML
1K23088_CK.EML
1K23094_CK.EML
1K23101_CK.EML
1K23104_CK.EML
1K24001_CK.EML
1K24020_CK.EML
1K24033_CK.EML
1K24059_CK.EML
1K24062_CK.EML
1K24107_CK.EML
1K24130_CK.EML
1K24137_CK.EML
1K24999_CK.EML
1K25031_CK.EML
1K25048_CK.EML
1K25072_CK.EML
1K25084_CK.EML
1K25112_CK.EML
1K25999_CK.EML
1K30999_CK.EML
1K31005_CK.EML
1K31033_CK.EML
1K32003_CK.EML
1K33011_CK.EML
1K33016_CK.EML
1K33021_CK.EML
1K33029_CK.EML
1K33037_CK.EML
1K33041_CK.EML
1K34003_CK.EML
1K34013_CK.EML
1K34022_CK.EML
1K34027_CK.EML
1K35005_CK

## Loop over files

In [145]:
def parse_kamer_file(filename):
    soup = BeautifulSoup(open(filename), "html.parser")
    contestname = soup.find('contestname').text
    niscode = soup.find('contestidentifier').attrs['idnumber']
    
    candidates = []
    partijen = []

    for reportingunitvotes in soup.findAll('reportingunitvotes'):
        partij = reportingunitvotes.reportingunitidentifier.text

        for i, selection in enumerate(soup.findAll('selection')):
            if selection.find('registeredname'): 
                #print(selection)
                temp_dict = {'name':selection.registeredname.text, 
                                'validvotes':selection.validvotes.text, 
                            'niscode':niscode,
                            'kanton':contestname}
                for countmetric in selection.findAll('countmetric'):
                    temp_dict[countmetric.attrs['type']] = countmetric.text
                partijen.append(temp_dict)

                continue
            elif selection.find('candidate'):
                candidates.append({'name': selection.candidate.knownas.text, 
                                  'votes': selection.validvotes.text, 
                                  'partij':partij, 
                                  'niscode': niscode, 
                                  'kanton': contestname})
            #else:
                #print(selection)

    # Zitten blijkbaar dubbels in de partijen, elke record zit er 10 keer in?
    partijen = pd.DataFrame(partijen).drop_duplicates().to_dict(orient='records')
    candidates = pd.DataFrame(candidates).drop_duplicates().to_dict(orient='records')    
                
    return candidates, partijen

In [151]:
candidates_total = []
partijen_total = []
for i, file in enumerate(interesting_files):
    print(i)
    candidates, partijen = parse_kamer_file('data_verkiezingsuitslagen/xmls/{}'.format(file))
    candidates_total.extend(candidates)
    partijen_total.extend(partijen)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221


## Convert to CSV and save

In [157]:
candidates_total = pd.DataFrame(candidates_total)
candidates_total.to_csv('data_verkiezingsuitslagen/candidates_total.csv', index=False)

In [158]:
partijen_total = pd.DataFrame(partijen_total)
partijen_total.to_csv('data_verkiezingsuitslagen/partijen_total.csv', index=False)